In [ ]:
import sqlite3

path = 'database.db'
connection = sqlite3.connect(path)
cursor = connection.cursor()

In [ ]:
with sqlite3.connect('database.db') as connection:
    cursor = connection.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS inference_v0_output (
            preview STRING,
            label STRING
        )
    ''')
    connection.commit()
    cursor.close()

In [ ]:
import pandas as pd

with sqlite3.connect('database.db') as connection:

    sql_query = 'SELECT * FROM inference_v0_output'

    df = pd.read_sql(sql_query, connection)


df.head(3)

In [ ]:
with sqlite3.connect('database.db') as connection:
    cursor = connection.cursor()
    cursor.execute('''
        DROP TABLE inference_v0_output
    ''')
    connection.commit()
    cursor.close()

In [ ]:
import pandas as pd

with sqlite3.connect('database.db') as connection:

    sql_query = 'SELECT * FROM inference_v0_LLM_output'

    df_LLM = pd.read_sql(sql_query, connection)


df_LLM

In [ ]:
full_df = df.join(df_LLM, rsuffix='_llm')
full_df['preview_diff'] = full_df['preview'] != full_df['preview_llm']
full_df['label_diff'] = full_df['label'] != full_df['label_llm']
full_df

In [ ]:
import difflib

changes = []

for _, content in full_df[full_df['preview_diff']][['preview', 'preview_llm']].iterrows():
    print(content['preview'])
    print(content['preview_llm'])
    changes += ([change[2:] for change in difflib.ndiff(content['preview'], content['preview_llm']) if change[0] in ('+', '-')])

changes_dict = {}

for item in changes:
    if item in changes_dict:
        changes_dict[item] += 1
    else:
        changes_dict[item] = 1

print(dict(sorted(changes_dict.items(), key=lambda x: x[1])))